In [1]:
import os
import json
import pandas as pd
import traceback

In [15]:
from langchain_openai import ChatOpenAI

In [12]:
KEY=os.getenv("OPENAI_API_KEY")

In [10]:
from dotenv import load_dotenv
load_dotenv()

True

In [16]:
llm=ChatOpenAI(openai_api_key=KEY,model="gpt-3.5-turbo",temperature=0.5)